In [ ]:
import yfinance as yf 
import pandas as pd
from matplotlib import pylab as plt
import numpy as np
from datetime import datetime



import quandl


# Functions


In [ ]:
def import_MSCI(name):
    '''Function to clean data from MSCI
    https://www.msci.com/end-of-day-data-country
    '''
    
    
    path = "../Data/" + name + ".xls"
    imp_tmp = pd.read_excel(path)
    price_type = imp_tmp.loc[1][1]
    
    imp = imp_tmp[6:]
    imp.columns = ['Date', name]
    #imp['Date'] = pd.to_datetime(imp['Date'], errors='coerce')
    imp.loc[0] = pd.to_datetime(imp['Date'], errors='coerce')
    imp = imp.dropna()
    
    imp = imp.set_index('Date')
    imp.index = imp.index.strftime('%Y/%m')
    imp.index = pd.to_datetime(imp.index, errors='coerce')
    
    print(price_type)
    
    return(imp)

def calc_bond_returns(YTM, YTMM1, M):
    DY = YTM - YTMM1
    yield_income = (1+YTM)**(1/12) - 1
    Z = (1+(YTM/2))
    

    duration = (1/YTM)* (1 -(1/(1+0.5*YTM)**(2*M)))


    C1 = (2/YTM**2) * (1-Z**(-2*M))
    C2 = ((2*M)/YTM) * Z **(-2*M - 1) 
    convex = C1-C2

    ret = yield_income - duration* DY + (1/2)*convex * (DY**2)
    return(ret)

def monthly_bonds_ret(bond_YTM, M):
    bond_ret = [0]

   
    
    for i in range(len(bond_YTM)-1):
        YTM = bond_YTM[i+1]
        YTMM1 = bond_YTM[i]
        
        ret = calc_bond_returns(YTM, YTMM1, M)
        bond_ret.append(ret)

    bond_ret = pd.DataFrame(bond_ret)
    bond_ret.index = bond_YTM.index + pd.Timedelta('1 day')
    
    bond_ret = (bond_ret+1).cumprod()
    return(bond_ret)

In [ ]:
def import_MSCI(name):
    '''Function to clean data from MSCI
    https://www.msci.com/end-of-day-data-country
    '''
    
    
    path = "../Data/" + name + ".xls"
    imp_tmp = pd.read_excel(path)
    price_type = imp_tmp.loc[1][1]
    
    imp = imp_tmp[6:]
    imp.columns = ['Date', name]
    imp.loc[0] = pd.to_datetime(imp['Date'], errors='coerce')
    imp = imp.dropna()
    imp = imp.set_index('Date')
    imp.index = imp.index.strftime('%Y/%m')
    imp.index = pd.to_datetime(imp.index, errors='coerce') # sets to first day of month 
    imp.index = imp.index.shift(1, freq='MS') # raw data is end of month, set to first day next month
    
    print(price_type)
    
    return(imp)

In [ ]:
def bond_return(filename, sheet_no=0, header=0):
    '''10 YEAR BOND RETURN. sheet1:Original US, sheet2: Improved US, sheet3:germany, 
    sheet4:Japan, sheet5:Australia, sheet6:Norway. sheet7:Sweden'''
    
    tab = ezodf.opendoc(filename="../Data/"+filename).sheets[sheet_no]
    temp_data= pd.DataFrame({col[header].value:[x.value for x in col[header+1:]]
                         for col in tab.columns()})
    temp_data['observation_date'] = pd.to_datetime(temp_data['observation_date'], errors='coerce')
    temp_data.index = temp_data['observation_date']
    bond_ret = pd.DataFrame(temp_data['Cum Ret'])
    bond_ret = bond_ret.dropna()
    bond_ret.columns = ['10Y Bond']
    return(bond_ret)

# Data Import

In [ ]:
# Trend Following
%run TF_Data.ipynb

In [1]:
### BOND DATA WITH OWN FUNCTION

T_Bill_tmp = yf.download('^IRX') #data import
T_Bill = T_Bill_tmp['Adj Close']
T_Bill = T_Bill.resample('M').last()/100

T_Bill = monthly_bonds_ret(T_Bill, 0.25)
T_Bill.columns = ['3 Month T-Bill']

bond_30YR_tmp = yf.download('^TYX') #data import
bond_30YR = bond_30YR_tmp['Adj Close']
bond_30YR_monthly = bond_30YR.resample('M').last()/100

Ret_bond_30YR = monthly_bonds_ret(bond_30YR_monthly, 30)
Ret_bond_30YR.columns = ['Bond 30 Yr']


bond_5YR_tmp = yf.download('^FVX') #data import
bond_5YR = bond_5YR_tmp['Adj Close']
bond_5YR_monthly = bond_5YR.resample('M').last()/100

Ret_bond_5YR = monthly_bonds_ret(bond_5YR_monthly, 5)
Ret_bond_5YR.columns = ['Bond 5 Yr']

bond_10YR_tmp = yf.download('^TNX') #data import
bond_10YR = bond_10YR_tmp['Adj Close']
bond_10YR_monthly = bond_10YR.resample('M').last()/100

Ret_bond_10YR = monthly_bonds_ret(bond_10YR_monthly, 10)
Ret_bond_10YR.columns = ['Bond 10 Yr']

NameError: name 'yf' is not defined

In [ ]:
# SP500
SP500_daily_tmp = yf.download("^SP500TR")
SP500_daily= pd.DataFrame(SP500_daily_tmp['Close'])
SP500_daily.columns = ['SP500']

#USA
USA_monthly = import_MSCI('USA')

# Gold
Gold_tmp = quandl.get("LBMA/GOLD", authtoken="6rNSyTM5R4g7_B7XSyQW")
Gold_daily = pd.DataFrame(Gold_tmp["USD (AM)"])
Gold_daily.columns = ['Gold']

Gold_monthly = Gold_daily.resample('M').last()
Gold_monthly.columns = ['Gold']
Gold_monthly.index = Gold_monthly.index + pd.Timedelta('1 day')

# 3 year US Government Bond
SHY_daily_tmp = yf.download("SHY")
SHY_daily = pd.DataFrame(SHY_daily_tmp['Close'])
SHY_daily.columns = ['SHY']

SHY_monthly = SHY_daily.resample('M').last()
SHY_monthly.index = SHY_monthly.index + pd.Timedelta('1 day')

# 10 year US Government bond
path = 'International Monthly Government Bond Returns.ods'
Bond_monthly = bond_return(filename=path, sheet_no = 2)
Bond_monthly.index = Bond_monthly.index + pd.Timedelta('1 day')


# 20 year US Treasury Bond
TLT_daily_tmp =  yf.download("TLT")
TLT_daily = pd.DataFrame(TLT_daily_tmp['Close'])
TLT_daily.columns = ['TLT']

TLT_monthly = TLT_daily.resample('M').last()
TLT_monthly.index = TLT_monthly.index + pd.Timedelta('1 day')




gold_miners_tmp = yf.download('^HUI')
gold_miners = pd.DataFrame(gold_miners_tmp['Adj Close'])
gold_miners = gold_miners.resample('M').last()
gold_miners.columns = ['Gold Miner']
gold_miners.index = gold_miners.index + pd.Timedelta('1 day')

In [ ]:
multi_strat_imp = pd.read_excel(r"https://www.eurekahedge.com/Indices/ExportIndexReturnsToExcel?IndexType=Eurekahedge&IndexId=483")
multi_strat = multi_strat_imp[3:]
multi_strat.index= pd.to_datetime(multi_strat['Unnamed: 0'])
multi_strat = pd.DataFrame(multi_strat['Unnamed: 2'])
multi_strat.columns = ["Multi Strategy"]

# Long Vol
long_vol_imp = pd.read_excel (r'https://www.eurekahedge.com/Indices/ExportIndexReturnsToExcel?IndexType=Eurekahedge&IndexId=640')

long_vol = long_vol_imp[3:]
long_vol.index= pd.to_datetime(long_vol_imp[3:]['Unnamed: 0'])
long_vol = pd.DataFrame(long_vol['Unnamed: 2'])
long_vol.columns = ["Long Vol"]



In [ ]:
USA = import_MSCI('USA')
RUSSIA = import_MSCI('RUSSIA')
SWEDEN = import_MSCI('SWEDEN')
Emerging_Markets = import_MSCI('Emerging_Markets')
JAPAN = import_MSCI('JAPAN')
PACIFIC_ex_JAPAN = import_MSCI('PACIFIC_ex_JAPAN')
small_cap = import_MSCI('SWEDISH_SMALL')